# NBA Playoff Predictor - Data Cleaning

This notebook focuses on cleaning and preprocessing the raw NBA data from Kaggle sources. We'll prepare the data for feature engineering by standardizing formats, handling missing values, and ensuring data quality across different sources.

## Data Sources and Cleaning Goals

1. **NBA/ABA/BAA Stats (sumitrodatta)**
   - Player Season Info: Contains individual player statistics per season
     - Cleaning focuses on standardizing team names, filtering for NBA-only data
     - Adding conference information for each team
     - Handling missing values
   - Team Stats Per Game: Contains team-level performance metrics
     - Cleaning involves normalizing team names
     - Adding conference mappings
     - Ensuring consistent statistical calculations

2. **NBA Injury Stats (loganlauton)**
   - Contains historical injury data from 1951-2023
   - Cleaning involves:
     - Standardizing team names
     - Converting dates to consistent format
     - Adding conference information
     - Creating yearly injury summaries per team
     - Removing duplicate entries

## Conference Mapping

Each dataset is enhanced with conference information to support playoff prediction:
- Teams are mapped to either 'EAST' or 'WEST' conference
- Historical conference alignments are considered
- Conference information is crucial for playoff qualification rules
  - Pre-2020: Top 8 teams per conference
  - 2020 onward: Top 6 automatic + Play-in tournament (7-10)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from pathlib import Path

sys.path.append('..')

from src.data.cleaners.nba_data_cleaner import NBACleaner
from src.data.utils import setup_logging

logger = setup_logging()

sns.set_theme()

cleaner = NBACleaner()

## Clean NBA/ABA/BAA Stats

Process data from sumitrodatta's dataset.

The cleaning process ensures consistent formatting, adds conference information, and removes any anomalies that could affect our analysis.

# Player Season Data Cleaning Script

This script is designed to clean and standardize NBA player season data for analysis. It performs the following operations:

1. **Load Data**: Reads the raw player season data from a CSV file.
2. **Filter by Season**: Removes records from seasons prior to 2004 to focus on recent data.
3. **Drop Unnecessary Columns**: Eliminates irrelevant columns such as `player_id`, `seas_id`, `birth_year`, `pos`, and `lg`.
4. **Standardize Names**:
   - Player names are standardized using `NBACleaner`
   - Team names are converted to uppercase and standardized
5. **Add Conference Information**:
   - Maps teams to their respective conferences
   - Handles historical conference changes
6. **Convert Columns**:
   - The `age` column is converted to an integer type
   - Other numeric columns are standardized as needed
7. **Validate Data**:
   - Checks for missing values
   - Verifies conference mappings

### Output
- A cleaned dataset is saved as `../data/processed/player_season.csv`
- Includes player statistics with team and conference information

In [ ]:
# Load and clean player season info data
logger.info("Loading player season info data...")
ps_df_raw = pd.read_csv('../data/raw/kaggle/sumitrodatta/nba-aba-baa-stats/Player Season Info.csv')
logger.info(f"Initial player season info records: {len(ps_df_raw):,}")

# Make a copy of the raw data for processing
ps_df_processed = ps_df_raw.copy()

# Filter out data before 2004
logger.info("Filtering out records before 2004...")
ps_df_processed = ps_df_processed[ps_df_processed['season'] >= 2004]
logger.info(f"Records after filtering pre-2004 data: {len(ps_df_processed):,}")

# Drop unnecessary columns
columns_to_drop = ['player_id', 'seas_id', 'birth_year', 'pos', 'lg']
logger.info(f"Dropping unnecessary columns: {columns_to_drop}...")
ps_df_processed.drop(columns=columns_to_drop, inplace=True)
logger.info("Unnecessary columns dropped.")

# Standardize player names
logger.info("Standardizing player names...")
ps_df_processed = cleaner.standardize_player_names(ps_df_processed)
logger.info("Player name standardization complete.")

# Standardize team names
logger.info("Standardizing team names: converting to uppercase and stripping whitespace...")
ps_df_processed['tm'] = ps_df_processed['tm'].str.strip().str.upper()
ps_df_processed = cleaner.standardize_team_names(ps_df_processed, ['tm'])
logger.info("Team name standardization complete.")

# Rename 'tm' column to 'team'
logger.info("Renaming 'tm' column to 'team'...")
ps_df_processed.rename(columns={'tm': 'team'}, inplace=True)
logger.info("Column 'tm' successfully renamed to 'team'.")

# Convert 'age' column to integer type
logger.info("Converting 'age' column to integer type...")
ps_df_processed['age'] = pd.to_numeric(ps_df_processed['age'], errors='coerce').fillna(0).astype(int)
logger.info("'age' column conversion to integer complete.")

# Add conference mappings
ps_df_processed = cleaner.add_conference_mappings(ps_df_processed, name_col='team')

unknown_teams = ps_df_processed[ps_df_processed['conference'] == 'Unknown']['team'].unique()
if len(unknown_teams) > 0:
    logger.warning(f"Found teams with unknown conference: {unknown_teams}")
else:
    logger.info("All teams successfully mapped to conferences")

# Check for NaN values
logger.info("Checking for NaN values in the dataset...")
nan_cols = ps_df_processed.columns[ps_df_processed.isna().any()].tolist()
if nan_cols:
    logger.warning(f"Found NaN values in the following columns: {nan_cols}")
    for col in nan_cols:
        nan_count = ps_df_processed[col].isna().sum()
        logger.warning(f"Column '{col}' has {nan_count:,} NaN values.")
else:
    logger.info("No NaN values found in the cleaned DataFrame.")

# Display sample and save
print("\nSample of cleaned data:")
print(ps_df_processed.head())

output_path = '../data/processed/player_season.csv'
ps_df_processed.to_csv(output_path, index=False)
logger.info(f"Cleaned data saved to {output_path}")

In [ ]:
# Data for Player Season Data
dataset_name = "Player Season Data"
raw_count = len(ps_df_raw)  
processed_count = len(ps_df_processed) 

# Data for plotting
stages = ['Raw Data (Combined)', 'Processed Data']
counts = [raw_count, processed_count]
colors = ['steelblue', 'orange']

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 4))

# Create horizontal bars
bars = ax.barh(stages, counts, color=colors)

# Add value labels
for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height() / 2,
            f'{int(width):,}',
            ha='left', va='center', fontweight='bold')

# Customize the chart
ax.set_title(f'Data Cleaning Funnel: {dataset_name}', pad=20)
ax.set_xlabel('Number of Records')

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add padding
ax.set_xlim(0, max(counts) * 1.15)

plt.tight_layout()
plt.show()

# Team Statistics Data Cleaning Script

This script processes and cleans NBA team statistics data for analysis. It performs the following steps:

1. **Load Data**: Reads the raw team statistics data.
2. **Filter by Season**: Removes records prior to 2004.
3. **Remove League Averages**: Excludes league average entries.
4. **Standardize Team Names**:
   - Converts to uppercase
   - Maps to standardized codes
5. **Add Conference Information**:
   - Maps teams to conferences
   - Validates conference assignments
6. **Convert Data Types**:
   - Handles numeric columns
   - Converts percentages to decimals
7. **Validate Data**:
   - Checks for missing values
   - Verifies data consistency

### Output
- Cleaned dataset saved as `../data/processed/team_stats.csv`
- Includes team statistics with conference information

In [ ]:
# Load and clean team stats data
logger.info("Loading team stats data...")
ts_df_raw = pd.read_csv('../data/raw/kaggle/sumitrodatta/nba-aba-baa-stats/Team Stats Per Game.csv')
logger.info(f"Initial team stats records: {len(ts_df_raw):,}")

# Make a copy of the raw data for processing
ts_df_processed = ts_df_raw.copy()

# Filter out data before 2004
logger.info("Filtering out records before 2004...")
ts_df_processed = ts_df_processed[ts_df_processed['season'] >= 2004]
logger.info(f"Records after filtering pre-2004 data: {len(ts_df_processed):,}")

# Remove League Average entries
logger.info("Removing 'League Average' entries from the data...")
ts_df_processed = ts_df_processed[~ts_df_processed['team'].str.contains('League Average', case=False, na=False)]
logger.info(f"Records after removing 'League Average' entries: {len(ts_df_processed):,}")

# Standardize team names
logger.info("Standardizing team names...")
ts_df_processed['team'] = ts_df_processed['team'].str.strip().str.upper()
ts_df_processed = cleaner.standardize_team_names(ts_df_processed, ['team'])
logger.info("Team name standardization complete.")

# Convert percentage strings to decimals
logger.info("Converting percentage strings to decimal values...")
ts_df_processed = cleaner.convert_percentages(ts_df_processed)
logger.info("Percentage conversion complete.")

# Add conference mappings
ts_df_processed = cleaner.add_conference_mappings(ts_df_processed, name_col='team')

unknown_teams = ts_df_processed[ts_df_processed['conference'] == 'Unknown']['team'].unique()
if len(unknown_teams) > 0:
    logger.warning(f"Found teams with unknown conference: {unknown_teams}")
else:
    logger.info("All teams successfully mapped to conferences")

# Check for NaN values
logger.info("Checking for NaN values in the dataset...")
nan_cols = ts_df_processed.columns[ts_df_processed.isna().any()].tolist()
if nan_cols:
    logger.warning(f"Found NaN values in the following columns: {nan_cols}")
    for col in nan_cols:
        nan_count = ts_df_processed[col].isna().sum()
        logger.warning(f"Column '{col}' has {nan_count:,} NaN values.")
else:
    logger.info("No NaN values found in the cleaned DataFrame.")

# Display sample and save
print("\nSample of cleaned data:")
print(ts_df_processed.head())

output_path = '../data/processed/team_stats.csv'
ts_df_processed.to_csv(output_path, index=False)
logger.info(f"Cleaned data saved to {output_path}")

In [ ]:
# Data for Team Statistics data
dataset_name = "Team Statistics data"
raw_count = len(ts_df_raw)
processed_count = len(ts_df_processed)

# Data for plotting
stages = ['Raw Data (Combined)', 'Processed Data']
counts = [raw_count, processed_count]
colors = ['steelblue', 'orange']

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 4))

# Create horizontal bars
bars = ax.barh(stages, counts, color=colors)

# Add value labels
for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height() / 2,
            f'{int(width):,}',
            ha='left', va='center', fontweight='bold')

# Customize the chart
ax.set_title(f'Data Cleaning Funnel: {dataset_name}', pad=20)
ax.set_xlabel('Number of Records')

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add padding
ax.set_xlim(0, max(counts) * 1.15)

plt.tight_layout()
plt.show()

# Injury Data Cleaning and Summary Script

This script processes NBA player injury data to create team-level injury summaries. It performs the following steps:

1. **Load Data**: Reads raw injury data.
2. **Clean Dates**: Converts to datetime format.
3. **Filter Data**: Removes pre-2004 records.
4. **Standardize Teams**:
   - Converts team names to standard format
   - Maps historical team names
5. **Add Conference Information**:
   - Maps teams to conferences
   - Validates conference assignments
6. **Create Summary**:
   - Groups by year and team
   - Counts injuries per team-season

### Outputs
- Injury summary: `../data/processed/injuries_summary.csv`
  - Year, team, conference, injury count

In [ ]:
# Load and clean injury data
logger.info("Loading player injury data...")
injury_df_raw = pd.read_csv('../data/raw/kaggle/loganlauton/nba-injury-stats-1951-2023/NBA Player Injury Stats(1951 - 2023).csv')
logger.info(f"Initial player injury records: {len(injury_df_raw):,}")

# Make a copy of the raw data for processing
injury_df_processed = injury_df_raw.copy()

# Convert dates to datetime
logger.info("Converting 'Date' column to datetime format...")
injury_df_processed = cleaner.handle_dates(injury_df_processed, ['Date'])
logger.info("Date conversion complete.")

# Filter out data before 2004
logger.info("Filtering out records before 2004...")
injury_df_processed = injury_df_processed[injury_df_processed['Date'] >= '2004-01-01']
logger.info(f"Records after filtering pre-2004 data: {len(injury_df_processed):,}")

# Drop unnecessary columns
columns_to_drop = ['Unnamed: 0', 'Acquired', 'Relinquished', 'Notes']
logger.info(f"Dropping unnecessary columns: {columns_to_drop}...")
injury_df_processed.drop(columns=columns_to_drop, inplace=True)
logger.info("Unnecessary columns dropped.")

# Standardize team names
logger.info("Standardizing team names...")
injury_df_processed['Team'] = injury_df_processed['Team'].str.strip().str.upper()
injury_df_processed = cleaner.standardize_team_names(injury_df_processed, ['Team'])
logger.info("Team name standardization complete.")

# Extract year and create summary
logger.info("Creating injury summary...")
injury_df_processed['Year'] = injury_df_processed['Date'].dt.year
injury_summary_df = injury_df_processed.groupby(['Year', 'Team']).size().reset_index(name='Count')

# Add conference mappings
injury_summary_df = cleaner.add_conference_mappings(injury_summary_df, name_col='Team')

unknown_teams = injury_summary_df[injury_summary_df['conference'] == 'Unknown']['Team'].unique()
if len(unknown_teams) > 0:
    logger.warning(f"Found teams with unknown conference: {unknown_teams}")
else:
    logger.info("All teams successfully mapped to conferences")

# Standardize column names
logger.info("Converting column names to lowercase...")
injury_summary_df.columns = injury_summary_df.columns.str.lower()

# Display sample and save
print("\nSample of injury summary:")
print(injury_summary_df.head())

output_path = '../data/processed/injuries_summary.csv'
injury_summary_df.to_csv(output_path, index=False)
logger.info(f"Injury summary saved to {output_path}")

In [ ]:
# Data for Player Injury Data
dataset_name = "Player Injury Data"
raw_count = len(injury_df_raw)  
processed_count = len(injury_df_processed) 

# Data for plotting
stages = ['Raw Data (Combined)', 'Processed Data']
counts = [raw_count, processed_count]
colors = ['steelblue', 'orange']

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 4))

# Create horizontal bars
bars = ax.barh(stages, counts, color=colors)

# Add value labels
for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height() / 2,
            f'{int(width):,}',
            ha='left', va='center', fontweight='bold')

# Customize the chart
ax.set_title(f'Data Cleaning Funnel: {dataset_name}', pad=20)
ax.set_xlabel('Number of Records')

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add padding
ax.set_xlim(0, max(counts) * 1.15)

plt.tight_layout()
plt.show()